# LLM 

In [ ]:
request_content = """Dal PDF fornito, estrai i valori di F1, F2, F3 e restituiscili esclusivamente in formato JSON. 
        Il JSON deve avere esattamente questa struttura:
        {
            "f1_kwh": x_kwh,
            "f2_kwh": y_kwh,
            "f3_kwh": z_kwh,
            "f1_kvarh_capacitive": x1_kvarh,
            "f1_kvarh_inductive": x2_kvarh,
            "f1_kvarh_injected": x3_kvarh,
            "f2_kvarh_capacitive": y1_kvarh,
            "f2_kvarh_inductive": y2_kvarh,
            "f2_kvarh_injected": y3_kvarh,
            "f3_kvarh_capacitive": z1_kvarh,
            "f3_kvarh_inductive": z2_kvarh,
            "f3_kvarh_injected": z3_kvarh,
            "error_msg": ""
        }
        Regole di estrazione dei dati:
        * f*_kwh:
            * Prendi i valori dalla colonna "Quantità" nella sezione "Consumo fatturato" del PDF.
            * Ogni fascia (F1, F2, F3) deve avere il proprio valore.
        * f*_kvarh_capacitive, f*_kvarh_inductive, f*_kvarh_injected:
            * Estrarre i valori direttamente dalle seguenti sezioni del PDF per ciascuna fascia:
                * f*_kvarh_capacitive → Consumo rilevato energia reattiva capacitiva
                * f*_kvarh_inductive → Consumo rilevato energia reattiva induttiva
                * f*_kvarh_injected → Consumo fatturato energia reattiva immessa
            * Non eseguire somme o calcoli: usa il valore riportato nel PDF.
            * Tutti i valori devono corrispondere alla stessa fascia oraria (F1, F2, F3).
        Se uno o più dati non sono reperibili, restituisci tutti i campi null, con questo JSON:
        {
            "f1_kwh": null,
            "f2_kwh": null,
            "f3_kwh": null,
            "f1_kvarh_capacitive": null,
            "f1_kvarh_inductive": null,
            "f1_kvarh_injected": null,
            "f2_kvarh_capacitive": null,
            "f2_kvarh_inductive": null,
            "f2_kvarh_injected": null,
            "f3_kvarh_capacitive": null,
            "f3_kvarh_inductive": null,
            "f3_kvarh_injected": null,
            "error_msg": "FAILED TO READ CONSUMPTIONS FROM ALLEDGED BILL FILE"
        }
        ATTENZIONE:
        * La risposta deve contenere solo il JSON, nessun commento, spiegazione o testo aggiuntivo
        * I nomi dei campi devono essere esattamente quelli indicati sopra"""

In [1]:
request_content = """Dal PDF fornito, estrai i valori di F1, F2, F3 e restituiscili esclusivamente in formato JSON. 
        Il JSON deve avere esattamente questa struttura:
        {
            "f1_kwh": x_kwh,
            "f2_kwh": y_kwh,
            "f3_kwh": z_kwh,
            "f1_kvarh_capacitive": x1_kvarh,
            "f2_kvarh_capacitive": y1_kvarh,
            "f3_kvarh_capacitive": z1_kvarh,
            "f1_kvarh_inductive": x2_kvarh,
            "f2_kvarh_inductive": y2_kvarh,
            "f3_kvarh_inductive": z2_kvarh,
            "f1_kvarh_injected": x3_kvarh,
            "f2_kvarh_injected": y3_kvarh,
            "f3_kvarh_injected": z3_kvarh,
            "error_msg": ""
        }
        Regole di estrazione dei dati:
        * f*_kwh:
            * Prendi i valori dalla colonna "Quantità" nella sezione "Consumo fatturato" del PDF.
            * Ogni fascia (F1, F2, F3) deve avere il proprio valore.

        * f*_kvarh_capacitive:
            * Estrarre i valori per ogni fascia (F1, F2, F3) direttamente dalla tabella "Consumo rilevato energia reattiva capacitiva" del PDF.
        * f*_kvarh_inductive:
            * Estrarre i valori per ogni fascia (F1, F2, F3) direttamente dalla tabella "Consumo rilevato energia reattiva induttiva" del PDF.
        * f*_kvarh_injected:
            * dalla tabella "Consumo fatturato energia reattiva immessa" del PDF, colonna F1, F2, F3, estrarre i valori per ogni fascia.

        Regole speciali:
        * Non eseguire somme o calcoli: usa il valore riportato nel PDF.
        * Se la tabella contiene solo una fascia, assegna il valore solo a quella fascia e imposta le altre a null.
        * NON usare il totale per riempire F1, F2, F3.
        Se uno o più dati non sono reperibili, restituisci tutti i campi null, con questo JSON:
        {
            "f1_kwh": null,
            "f2_kwh": null,
            "f3_kwh": null,
            "f1_kvarh_capacitive": null,
            "f2_kvarh_capacitive": null,
            "f3_kvarh_capacitive": null,
            "f1_kvarh_inductive": null,
            "f2_kvarh_inductive": null,
            "f3_kvarh_inductive": null,
            "f1_kvarh_injected": null,
            "f2_kvarh_injected": null,
            "f3_kvarh_injected": null,
            "error_msg": "FAILED TO READ CONSUMPTIONS FROM ALLEDGED BILL FILE"
        }
        ATTENZIONE:
        * La risposta deve contenere solo il JSON, nessun commento, spiegazione o testo aggiuntivo
        * I nomi dei campi devono essere esattamente quelli indicati sopra"""

### AUTENTICAZIONE

In [2]:
import requests


BASE_URL = "http://localhost:3000"
url = f"{BASE_URL}/api/v1/auths/signin"
payload = {
    "email": "eleonora.fabris@dbagroup.it", 
    "password": "Scoiattolo1!"
}
response = requests.post(url, json=payload)
token = response.json()["token"]


In [3]:
url = f"{BASE_URL}/api/v1/models"
JWT_TOKEN = token

HEADERS_JSON = {
    "Authorization": f"Bearer {JWT_TOKEN}",
    "Content-Type": "application/json"
}
HEADERS_FILE = {
    "Authorization": f"Bearer {JWT_TOKEN}"
}

params = {
        "max_new_tokens": 50, 
        "temperature": 0.7, # se 0.0 risponde sempre allo stesso modo, estremamente preciso
        "top_p": 0.9, # indica la variabilità del vocabolario
        # "repeat_penality": 0.7 # nessuna penalizzazione
    }

response = requests.get(url, headers=HEADERS_JSON)

if response.status_code == 200:
    print("Autenticazione riuscita!")
    print(response.json()) # lista modelli disponibili
else:
    raise Exception(f"Errore nell'autenticazione: {response.status_code}")



Autenticazione riuscita!
{'data': [{'id': 'gemma3:4b', 'name': 'gemma3:4b', 'object': 'model', 'created': 1767887009, 'owned_by': 'ollama', 'ollama': {'name': 'gemma3:4b', 'model': 'gemma3:4b', 'modified_at': '2025-12-22T16:59:15.153912+01:00', 'size': 3338801804, 'digest': 'a2af6cc3eb7fa8be8504abaf9b04e88f17a119ec3f04a3addf55f92841195f5a', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'gemma3', 'families': ['gemma3'], 'parameter_size': '4.3B', 'quantization_level': 'Q4_K_M'}, 'connection_type': 'local', 'urls': [0]}, 'connection_type': 'local', 'tags': [], 'actions': [], 'filters': []}, {'id': 'falcon3:7b', 'name': 'falcon3:7b', 'object': 'model', 'created': 1767887009, 'owned_by': 'ollama', 'ollama': {'name': 'falcon3:7b', 'model': 'falcon3:7b', 'modified_at': '2025-12-22T16:51:27.8742221+01:00', 'size': 4569738977, 'digest': '472ea1c89f64f932368963bfb8c7f54bd797f2161932cf0a72766a587b2e586d', 'details': {'parent_model': '', 'format': 'gguf', 'family': 'llama', 'families

### CARICAMENTO FILE

In [4]:
import requests
import json


pdf_file_path = r"C:\DEV\LAB\electrical-energy-billing\Fattura_0000005222062506.pdf"
with open(pdf_file_path, "rb") as f:
    files = {"file": ("Fattura.pdf", f, "application/pdf")}
    response = requests.post(f"{BASE_URL}/api/v1/files/?process=true", files=files, headers=HEADERS_FILE)

if response.status_code not in [200, 201]:
    raise Exception(f"Errore nel caricamento del file: {response.status_code} - {response.text}")

file_data = response.json()
file_id = file_data.get("id")
# print("File caricato:", file_id)

with requests.get(
    f"{BASE_URL}/api/v1/files/{file_id}/process/status?stream=true",
    headers=HEADERS_FILE,
    stream=True
) as r:
    if r.status_code != 200:
        raise Exception(f"Errore status SSE (Server-Sent Events): {r.status_code} - {r.text}")
    for line in r.iter_lines(decode_unicode=True):
        if not line:
            continue
        if line.startswith("data:"):
            data = json.loads(line.replace("data:", "").strip())
            status = data.get("status")
            # print("Status:", status)
            if status == "completed":
                print(f"Caricamento del file {file_id} completato.")
                break
            if status in ("failed", "error"):
                raise Exception("Processing fallito")

Caricamento del file e55e21e6-e8f1-44c9-a3ce-fd8d5c4729a7 completato.


### CREAZIONE NUOVA CHAT

In [5]:
import requests
import uuid


initial_assistant_id = str(uuid.uuid4())

payload_new_chat = {
    "chat": {
        "title": "Lettura della bolletta",
        "models": ["falcon3:7b"],
        "params": params
    }
}
url_new_chat = f"{BASE_URL}/api/v1/chats/new"
response = requests.post(url_new_chat, json=payload_new_chat, headers=HEADERS_JSON)
if response.status_code not in [200, 201]:
    raise Exception(f"Errore nella creazione chat: {response.status_code} - {response.text}")

chat_data = response.json()
chat_id = chat_data["id"]
current_id = initial_assistant_id
print("Nuova chat creata:", chat_id)


Nuova chat creata: fff38876-1964-430f-9521-94a0555f9cb7


In [6]:
url_get_chat = f"{BASE_URL}/api/v1/chats/{chat_id}"
chat_info = requests.get(url_get_chat, headers=HEADERS_JSON).json()
# chat_info

### INVIO DEL PROMPT

In [7]:
import uuid
import time

# assistant_msg_id = str(uuid.uuid4())
# files = [{"id": file_id}] if file_id else []

# payload_chat = {
#     "chat": {
#         "models": ["falcon3:7b"], # modello della chat
#         "history": {
#             "messages": {
#                 assistant_msg_id: {
#                     "id": assistant_msg_id,
#                     "parentId": None,
#                     "role": "assistant",
#                     "content": "", 
#                     "model": "falcon3:7b",
#                     "models": ["falcon3:7b"],
#                     "timestamp": int(time.time())
#                 }
#             },
#             "currentId": assistant_msg_id
#         }
#     },
#     "files": files , # file da allegare 
#     "params": {}
# }

# url_init_chat  = f"{BASE_URL}/api/v1/chats/{chat_id}"
# response = requests.post(url_init_chat, json=payload_chat, headers=HEADERS_JSON)
# response.raise_for_status()

# print("Chat inizializzata.")


In [ ]:
payload_completion = {
    "model": "falcon3:7b",
    "stream": False,
    "params": {
        "max_new_tokens": 50,
        "temperature": 0.5,
        "top_p": 0.85,
        "mirostat": 0
    },
    "messages": [
        {
            "role": "user",
            "content": request_content
        }
    ],
    "files": [
        {
            "id": file_id,
            "type": "file"
        }
    ]
}
url_completion = f"{BASE_URL}/api/chat/completions"
response = requests.post(url_completion, json=payload_completion, headers=HEADERS_JSON)
if response.status_code != 200:
    raise Exception(f"Errore completions: {response.status_code} - {response.text}")


### LEGGI LA RISPOSTA

In [38]:
completion_data = response.json()
answer = completion_data["choices"][0]["message"]["content"]

print(f"Risposta: {answer}")


Risposta: ```json
{
    "f1_kwh": 1150,
    "f2_kwh": 225,
    "f3_kwh": 300,
    "f1_kvarh_capacitive": 13,
    "f2_kvarh_capacitive": 7,
    "f3_kvarh_capacitive": 12,
    "f1_kvarh_inductive": 3,
    "f2_kvarh_inductive": 1,
    "f3_kvarh_inductive": 1,
    "f1_kvarh_injected": 325,
    "f2_kvarh_injected": 325,
    "f3_kvarh_injected": 0,
    "error_msg": ""
}
```


In [ ]:
url_completed = f"{BASE_URL}/api/chat/completed"
close_chat = requests.post(url_completed, headers=HEADERS_JSON)


### ELIMINA CHAT E FILE PDF

In [21]:
import subprocess
import requests

file_path = file_data["path"]

response = requests.delete(f"{BASE_URL}/api/v1/chats/{chat_id}", headers=HEADERS_FILE)

if response.status_code not in (200, 204):
    raise Exception(f"Errore cancellazione chat: {response.status_code} - {response.text}")
print(f"Chat {chat_id} eliminata correttamente.")

result = subprocess.run([
    "docker", "exec", "open-webui",
    "rm", "-f", file_path
], capture_output=True, text=True)

if result.returncode == 0:
    print(f"File PDF {file_id} eliminato correttamente.")
else:
    print("Errore eliminazione file:", result.stderr)


Chat 01758efc-b1a4-4734-9b0d-a52a050dd52b eliminata correttamente.
File PDF e52b3f19-cf08-4edd-89b8-9379595da66f eliminato correttamente.
